# Lectura 15: Parquet

## `read_parquet`

La primera función que vamos a estudiar es `read_parquet`, la cual nos va a permitir crear un DataFrame a partir de la lectura de un archivo parquet.

In [ ]:
import polars as pl

### Leer un solo archivo parquet

In [ ]:
df = pl.read_parquet('./data/parquet/datos.parquet')

In [ ]:
print(df)

#### Otra alternativa que leería todos los archivos parquet dentro de una carpeta

In [ ]:
df1 = pl.read_parquet('./data/multi_parquet/*.parquet')

In [ ]:
print(df1)

La forma recomendada de leer un archivo parquet cuando existan varios archivos parquet en una sola carpeta es utilizar el parámetro `use_pyarrow=True`. El propio Polars en su documentación expresa que esta opción es más estable que la que la opción de parquet reader que trae Rust que es la opción por defecto.

In [ ]:
df2 = pl.read_parquet('./data/multi_parquet/', use_pyarrow=True)

In [ ]:
print(df2)

### Leer un archivo parquet que esté particionado.
Aquí volvemos a usar el parámetro `use_pyarrow=True`.

In [ ]:
df3 = pl.read_parquet('./data/vuelos/', use_pyarrow=True)

In [ ]:
print(df3)

### Seleccionar solo un conjunto de columnas del archivo parquet
Si deseamos seleccionar un conjunto de columnas del archivo parquet que vamos a leer debemos usar el parámetro `columns`.

In [ ]:
df4 = pl.read_parquet('./data/vuelos/', use_pyarrow=True, columns=['YEAR', 'MONTH', 'DAY'])

print(df4)

## `scan_parquet`

Lee de forma laizy desde un archivo(o archivos) parquet. Estos archivos pueden estar alojados localmente o en la nube. Esta función permite que el optimizador de consultas baje los predicados y las proyecciones al nivel del escaneo, lo que generalmente aumenta el rendimiento y reduce la sobrecarga de memoria.

In [ ]:
df5 = pl.scan_parquet('./data/multi_parquet/*.parquet')

print(df5.collect())

## `read_parquet_schema`

Con esta función obtendremos un diccionario con el schema del archivo parquet sin necesidad de leerlo.

In [ ]:
schema = pl.read_parquet_schema('./data/parquet/datos.parquet')

schema

## `write_parquet`

Con esta función escribiremos un archivo parquet. Para este ejemplo usaremos el parámetro `use_pyarrow=True` lo cual utilizará la implementación de parquet C++ en vez de la implementación de parquet Rust.

In [ ]:
df.write_parquet('./data/salida.parquet', use_pyarrow=True)

A continuación vamos a escribir un parquet particionado, para ello vamos a utilizar el `df3` que previamente hemos creado y lo escribiremos particionado por la columna `MONTH`. Por último, cambiaremos el formato de compresión a snappy con el parámetro `compression='snappy'`.

Respecto a la compresión en su documentación Polars nos dice lo siguiente:
- Seleccionar `zstd` (opción por defecto) par un un buen desempeño de compresión.
- Seleccionar `lz4` para una rápida compresión/descompresión.
- Seleccionar `snappy` para garantizar una mayor compatibilidad con lectores de parquet más antigüos

In [ ]:
df3.write_parquet(
    './data/salida_particionada',
    use_pyarrow=True,
    pyarrow_options={"partition_cols": ['MONTH']},
    compression='snappy'
)

## `sink_parquet`

Esta función nos permitirá evaluar la query que define al lazyframe y escribirla en un archivo parquet. Esto permite que los resultados que sean más grandes que la memoria RAM puedan ser escribidos en disco.

In [ ]:
df5.sink_parquet('./data/salida_lazy.parquet')